Scraper based on Selenium

12-24-2017.

Rather than using a for loop and click next, I can directly call the javascript function

---

In [1]:
import string, time

In [2]:
import pandas as pd
from bs4 import BeautifulSoup as Soup

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
def replace_digit(my_str):
    new_text = list()
    for char in my_str:
        if char in string.digits:
            eval("new_text.append(Keys.NUMPAD%s)"%char)
        else:
            new_text.append(char)
    return(''.join(new_text))

In [5]:
def parse_page(tr_list):
    #tr_list = [Soup(tr.text, 'lxml') for tr in tr_list]
    tmp = list()
    for tr in tr_list:
        tmp.append(list())
        for i, td in enumerate(tr.find_all('td')):
            if i == 0:
                tmp[-1].append(base_url+td.a['href'])
            tmp[-1].append(td.text.strip())
    return pd.DataFrame(tmp, columns=column_names)

---

#### Input arg: start/end date

In [20]:
yyyy = 1945
from_text = '01/01/%s'%yyyy
to_text = '31/12/%s'%(yyyy+10)
from_text, to_text

('01/01/1945', '31/12/1955')

In [21]:
from_text = replace_digit(from_text)
to_text = replace_digit(to_text)
from_text, to_text

('\ue01a\ue01b/\ue01a\ue01b/\ue01b\ue023\ue01e\ue01f',
 '\ue01d\ue01b/\ue01b\ue01c/\ue01b\ue023\ue01f\ue01f')

---

Disable Image

In [ ]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images":2}
chromeOptions.add_experimental_option("prefs", prefs)

---

#### Start browser

In [6]:
#browser = webdriver.Chrome(chrome_options=chromeOptions)
browser = webdriver.Chrome()

In [7]:
url = 'https://treaties.un.org/Pages/AdvanceSearch.aspx?tab=UNTS&clang=_en'

In [8]:
browser.get(url)

---

#### Treaty type

In [9]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpSearchObj'))
select.select_by_visible_text('Treaty')

In [10]:
browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_rdbtreaty_2').click()

In [11]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))

In [12]:
select.select_by_index(20)

In [13]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$chkboxValues$2').click()

In [14]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$Adv_srch2').click()

In [15]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

---

#### Date

In [16]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))

In [17]:
select.select_by_visible_text('Date of Registration')

In [22]:
from_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtFrom')
from_field.send_keys(Keys.SPACE)
from_field.clear()
from_field.send_keys(from_text)
from_field.get_attribute('value')

'01/01/1945'

In [23]:
to_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtTo')
to_field.send_keys(Keys.SPACE)
to_field.clear()
to_field.send_keys(to_text)
to_field.get_attribute('value')

'31/12/1955'

In [24]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

In [25]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnSubmit').send_keys(Keys.SPACE)

---

#### Parse result

In [26]:
column_names = ['href', 'title', 'reg_num', 'reg_date', 'type', 'con_date', 'vol']
base_url = 'https://treaties.un.org'

In [27]:
df = pd.DataFrame(columns=column_names)

##### Test by paring 1 page

###### When page number is less than 10

In [121]:
pg = 7

In [122]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)

In [123]:
loc = int(pg%11) - 1
loc

6

In [124]:
tbody = browser.find_elements_by_tag_name('tbody')[3]
td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')
td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
print(td, len(td.find_all('a')))

<html><body><span>7</span></body></html> 0


###### When page > 10

In [125]:
pg = 26

In [126]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)

In [127]:
loc = int(pg%11) + (int(pg//11)+1)
loc

7

In [128]:
tbody = browser.find_elements_by_tag_name('tbody')[3]
td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')
td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
print(td, len(td.find_all('a')))

<html><body><a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$8')">8</a></body></html> 1


###### When page % 11 == 0

In [143]:
pg = 33

In [144]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)

In [145]:
loc = int(pg%11) + (int(pg//11)+1)
loc

4

In [168]:
tbody = browser.find_elements_by_tag_name('tbody')[3]
td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')
td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
print(td, len(td.find_all('a')))

<html><body><span>33</span></body></html> 0


###### Second to last page 

The second to last page seem to be a bit tricky

In [26]:
source = Soup(browser.page_source, 'lxml')

In [27]:
result_count = int(source.find('span', class_='RecordCount').text.split(':')[-1].strip())
result_count, result_count//10+1

(216, 22)

In [28]:
last_page = result_count//10+1
pg = last_page - 1
last_page, pg

(22, 21)

In [187]:
loc = int(pg%11) + (int(pg//11) + 1)
loc

10

In [188]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%(pg))

In [189]:
tbody = browser.find_elements_by_tag_name('tbody')[3]
td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')

In [190]:
len(td_list[2:-1])

12

In [193]:
td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
print(td, len(td.find_all('a')))

<html><body><span>39</span></body></html> 0


###### Last page 

In [194]:
pg = last_page

In [195]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%(pg))

In [196]:
loc = -1
loc

-1

In [197]:
tbody = browser.find_elements_by_tag_name('tbody')[3]
td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')

In [198]:
len(td_list[2:-1])

12

In [199]:
for td in td_list[2:-1]:
    print(td.get_attribute('innerHTML'))

<input type="image" src="../Images/Paging/first-btn.jpg" onclick="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$First');return false;" style="border-width:0px;">
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$30')">...</a>
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$31')">31</a>
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$32')">32</a>
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$33')">33</a>
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$34')">34</a>
<a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$35')">35</a>
<a href="javascript:__doPostBack('ctl00$ctl0

In [201]:
td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
print(td, len(td.find_all('a')))

<html><body><span>40</span></body></html> 0


##### Test iterate thru

In [36]:
df = pd.DataFrame(columns=column_names)

In [59]:
pg = 7
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)

In [61]:
pg = 20

In [62]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)
if pg > 10:
    loc = int(pg%11) + (int(pg//11)+1)
else:
    loc = int(pg%11) - 1
txt_ = "<span>%d</span>"%(pg)
while True:
    tbody = browser.find_elements_by_tag_name('tbody')[3]
    try:
        td_text = [td.get_attribute('innerHTML') for td in tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')]
    except:
        continue
    if txt_ in td_text:
        break
#time.sleep(pd.np.random.randint(2, 5))
tab = browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_dgTreaty')
tab_html = tab.get_attribute('innerHTML')
source = Soup(tab_html, 'lxml')
tbody = source.find('tbody')
tr_list = tbody.find_all('tr')[3:-2]
df = df.append(parse_page(tr_list), ignore_index=True)
print(pg, df.shape[0])
#time.sleep(pd.np.random.randint(6, 15))
pg += 1

20 90


In [ ]:
browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)
if pg > 10:
    loc = int(pg%11) + (int(pg//11)+1)
else:
    loc = int(pg%11) - 1
while True:
    try:
        tbody = browser.find_elements_by_tag_name('tbody')[3]
        td_list = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')
        td = Soup(td_list[2:-1][loc].get_attribute('innerHTML'), 'lxml')
        print(td, len(td.find_all('a')))
        time.sleep(pd.np.random.randint(2, 5))
        if len(td.find_all('a')) < 1:
            break 
    except:
        continue
tab = browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_dgTreaty')
tab_html = tab.get_attribute('innerHTML')
source = Soup(tab_html, 'lxml')
tbody = source.find('tbody')
tr_list = tbody.find_all('tr')[3:-2]
df = df.append(parse_page(tr_list), ignore_index=True)
print(pg, df.shape[0])
#time.sleep(pd.np.random.randint(6, 15))
pg += 1

<html><body><a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$21')">...</a></body></html> 1
<html><body><a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$21')">...</a></body></html> 1
<html><body><a href="javascript:__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$21')">...</a></body></html> 1


In [ ]:
df

In [63]:
browser.close()

---

#### Run

In [72]:
yyyy = 2020
interv = 20

In [73]:
from_text = '01/01/%s'%yyyy
to_text = '31/12/%s'%(yyyy+interv)
print(from_text, to_text)
from_text = replace_digit(from_text)
to_text = replace_digit(to_text)
from_text, to_text

01/01/2020 31/12/2040


('\ue01a\ue01b/\ue01a\ue01b/\ue01c\ue01a\ue01c\ue01a',
 '\ue01d\ue01b/\ue01b\ue01c/\ue01c\ue01a\ue01e\ue01a')

In [74]:
browser = webdriver.Chrome()
url = 'https://treaties.un.org/Pages/AdvanceSearch.aspx?tab=UNTS&clang=_en'

In [75]:
browser.get(url)

In [77]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpSearchObj'))
select.select_by_visible_text('Treaty')
browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_rdbtreaty_2').click()

In [78]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))
select.select_by_index(20)

In [79]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$chkboxValues$2').click()
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$Adv_srch2').click()
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

In [80]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))
select.select_by_visible_text('Date of Registration')

In [81]:
from_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtFrom')
from_field.send_keys(Keys.SPACE)
from_field.clear()
from_field.send_keys(from_text)

to_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtTo')
to_field.send_keys(Keys.SPACE)
to_field.clear()
to_field.send_keys(to_text)
to_field.get_attribute('value')

'31/12/2040'

In [82]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

In [83]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnSubmit').send_keys(Keys.SPACE)

Run the actual scraping

In [63]:
column_names = ['href', 'title', 'reg_num', 'reg_date', 'type', 'con_date', 'vol']
base_url = 'https://treaties.un.org'

In [64]:
source = Soup(browser.page_source, 'lxml')

In [65]:
result_count = int(source.find('span', class_='RecordCount').text.split(':')[-1].strip())
result_count, result_count/10, pd.np.ceil(result_count/10)

(354, 35.4, 36.0)

In [66]:
df = pd.DataFrame(columns=column_names)

In [67]:
#for pg in list(range(2, result_count//10+2)) + [1]:
for pg in list(range(2, int(pd.np.ceil(result_count/10))+1)) + [1]:
#for pg in list(range(21, result_count//10+2)) + [1]:
    browser.execute_script("__doPostBack('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty','Page$%d')"%pg)
    if pg > 10:
        loc = int(pg%11) + (int(pg//11)+1)
    elif pg == result_count//10+1:
        loc = -1
    else:
        loc = int(pg%11) - 1
    txt_ = "<span>%d</span>"%(pg)
    while True:
        try:
            tbody = browser.find_elements_by_tag_name('tbody')[3]
            td_text = [td.get_attribute('innerHTML') for td in tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')]
        except:
            continue
        if txt_ in td_text:
            break
    tab = browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_dgTreaty')
    tab_html = tab.get_attribute('innerHTML')
    source = Soup(tab_html, 'lxml')
    tbody = source.find('tbody')
    tr_list = tbody.find_all('tr')[3:-2]
    df = df.append(parse_page(tr_list), ignore_index=True)
    print(pg, df.shape[0])
    time.sleep(pd.np.random.randint(2, 6)+pd.np.random.random())

2 10
3 20
4 30
5 40
6 50
7 60
8 70
9 80
10 90
11 100
12 110
13 120
14 130
15 140
16 150
17 160
18 170
19 180
20 190
21 200
22 210
23 220
24 230
25 240
26 250
27 260
28 270
29 280
30 290
31 300
32 310
33 320
34 330
35 340
36 344
1 354


In [68]:
df.shape

(354, 7)

In [69]:
yyyy, yyyy+interv

(1999, 2019)

In [70]:
df.to_csv('./data/UNTC-%i-%i.csv'%(yyyy, yyyy+interv), index=False)

In [71]:
browser.close()